In [3]:
import scipy.stats as ss
from collections import OrderedDict
import numpy as np

In [4]:
# https://stackoverflow.com/questions/29762628/partition-of-a-set-or-all-possible-subgroups-of-a-list
def partitions(set_):
    if not set_:
        yield []
        return
    for i in range(int(2**len(set_)/2)):
        parts = [set(), set()]
        for item in set_:
            parts[i&1].add(item)
            i >>= 1
        for b in partitions(parts[1]):
            yield [parts[0]]+b

In [5]:
class Player:
    def __init__(self, name, score):
        self.name = name
        self.score = score
    def __repr__(self):
        return self.name
    def __lt__(self, other):
        # <= operator
        return self.score < other.score
    def __le__(self, other):
        # <= operator
        return self.score <= other.score

## Partition of set

Here's a all the possible coalition structures (all possible partition of the set $\{A, B, C\}$).

In [6]:
def get_partitions(players):
    old_all_partitions = list(partitions(players))
    all_partitions = []
    for coalition_struct in old_all_partitions:
        all_partitions.append(tuple(\
                frozenset(group) for group in coalition_struct))
    return all_partitions

In [7]:
A = Player('A', 3)
B = Player('B', 2)
C = Player('C', 1)

players = [A, B, C]
all_partitions = get_partitions(players)
all_partitions

[(frozenset({C, B, A}),),
 (frozenset({C, B}), frozenset({A})),
 (frozenset({C, A}), frozenset({B})),
 (frozenset({C}), frozenset({B, A})),
 (frozenset({C}), frozenset({A}), frozenset({B}))]

## Valuation Function of Game

If a coalition has at least 2 players, everyone gets a payoff of $2$ otherwise nothing.

In [8]:
def v(s):
    # valuation of a coalition
    return 5 * int(len(s) >= 2)

## Outcomes of each coalition structure

This code evaluates the outcome and relative ranking of each member for each coalition structure.

Here, we assume that these coalition structures are dictated by a central planner. The question of stability would be dealt with later

In [9]:
def get_outcomes_of_each_structure(all_partitions):
    struct_to_ranking = {} # key = coalition structure,
    # value = dict of ranking of players
    for coal_struct in all_partitions:
        actual_score = {}
        for group in coal_struct:
            reward = v(group)
            for player in group:
                actual_score[player.name] = player.score + reward
        # https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
        ranking = OrderedDict({k: v for k, v in 
                               sorted(actual_score.items(), key=lambda item: item[1])})
        struct_to_ranking[coal_struct] = ranking 
    return struct_to_ranking

In [10]:
struct_to_ranking = get_outcomes_of_each_structure(all_partitions)
struct_to_ranking

{(frozenset({C, B, A}),): OrderedDict([('C', 6), ('B', 7), ('A', 8)]),
 (frozenset({C, B}),
  frozenset({A})): OrderedDict([('A', 3), ('C', 6), ('B', 7)]),
 (frozenset({C, A}),
  frozenset({B})): OrderedDict([('B', 2), ('C', 6), ('A', 8)]),
 (frozenset({C}),
  frozenset({B, A})): OrderedDict([('C', 1), ('B', 7), ('A', 8)]),
 (frozenset({C}),
  frozenset({A}),
  frozenset({B})): OrderedDict([('C', 1), ('B', 2), ('A', 3)])}

In [33]:
d = OrderedDict({'a': 1, 'b': 2})
d['a'] = 3
print({k: v for k, v in 
                               sorted(d.items(), key=lambda item: item[1])})
print(d)

{'b': 2, 'a': 3}
OrderedDict([('a', 3), ('b', 2)])


In [11]:
group = frozenset({A,B,C})
list(filter(lambda x: group in x, struct_to_ranking))

[(frozenset({C, B, A}),)]

## Group stability

We now reason about group stability. A group $G$ is called stable if for each player $i \in G$ the best-worst vector of staying is no-worse than the best-worst vector of defecting to the open market $N \setminus G$.

In [24]:
# helper functions
def get_player_ranking_from_struct(player_name, struct_to_ranking,\
                               struct):
    return get_player_ranking(player_name, struct_to_ranking[struct])

def get_player_ranking(player_name, ranking):
    # e.g. 
    # input: ('A', OrderedDict([('C', 3), ('B', 4), ('A', 5)])
    # ==> 2
    print('ranking:', ranking)
    return list(ranking.keys()).index(player_name)

def best_worst_player(player_name, viable_structs, struct_to_ranking):
    best, worst = float('-inf'), float('inf')
    for coalition_struct in viable_structs:
        outcome = get_player_ranking_from_struct(player_name,\
                                    struct_to_ranking,
                                    coalition_struct)
        print('outcome:', outcome, 'for player:', player_name,
             'with struct:', coalition_struct, '\n')
        best = max(best, outcome)
        worst = min(worst, outcome)
    return best, worst

In [25]:
def is_group_stable(group, struct_to_ranking):
    '''
     group = frozenset({B, C})
     for each best-worst, entry
     write 
     1 - better to leave
     2 - better to stay
     0 - either way is equal!
     
    We assume that player would STAY if the
    worst-case of staying is STRICTLY better than the
    best-case of leaving!
    
    We assume that player would LEAVE if the 
    best-case of staying is STRICTLY worse than the
    worst-case of leaving!
    
    Otherwise, we don't know N/A
    '''
    stay = assume_player_stay(group, struct_to_ranking)
    leave = assume_player_leave(group, struct_to_ranking)
    res = np.zeros(len(group))
    for player in range(len(group)):
        best_stay, worst_stay = stay[0, player], stay[1, player]
        best_leave, worst_leave = leave[0, player], leave[1, player]
        if worst_stay > best_leave:
            res[player] = 2 #'stay'
        elif best_stay < worst_leave:
            res[player] = 1 #'leave'
        
    #res = np.zeros(shape=(2, len(group)))
    #res += 2 * (stay > leave)
    #res += 1 * (stay < leave)
    #
    s = {1: 'leave', 2: 'stay', 0:'N/A'}
    ## convert 1,2,0 to string
    return np.vectorize(lambda x: s[x])(res)

In [26]:
def assume_player_stay(group, struct_to_ranking):
    '''
    e.g. group = frozenset({B, C})
    k = number of players in group
    return 2 x k matrix
            player 1  player 2   ... player k
    best
    worst
    if staying
    '''
    best_worst = np.zeros(shape=(2, len(group)))
    # all the possible coalition structures containing G
    viable_structs = [struct for struct in struct_to_ranking\
                        if group in struct]
    print('viable staying:', viable_structs, '\n')
    for i, player in enumerate(group):
        best, worst = best_worst_player(player.name, viable_structs, struct_to_ranking) 
        print('>> best, worst STAY:', best, worst, 'for player:', player.name, '\n')
        best_worst[0, i] = best
        best_worst[1, i] = worst 
    return best_worst

In [27]:
group = frozenset({A, C})
assume_player_stay(group, struct_to_ranking)

viable staying: [(frozenset({C, A}), frozenset({B}))] 

ranking: OrderedDict([('B', 2), ('C', 6), ('A', 8)])
outcome: 1 for player: C with struct: (frozenset({C, A}), frozenset({B})) 

>> best, worst STAY: 1 1 for player: C 

ranking: OrderedDict([('B', 2), ('C', 6), ('A', 8)])
outcome: 2 for player: A with struct: (frozenset({C, A}), frozenset({B})) 

>> best, worst STAY: 2 2 for player: A 



array([[1., 2.],
       [1., 2.]])

In [55]:
def assume_player_leave(group, struct_to_ranking):
    '''
    k = number of players in group
    return 2 x k matrix
            player 1  player 2   ... player k
    best
    worst
    if leaving 
    '''
    best_worst = np.zeros(shape=(2, len(group)))
    for i, player in enumerate(group):
        new_group = set(group)
        new_group.remove(player)
        viable_structs = [struct for struct in struct_to_ranking\
                        if new_group in struct]
        print('viable structs:', viable_structs)
        new_struct_to_ranking = struct_to_ranking.copy()
        # modify new_struct_to_ranking, every one in this group
        # get 0
        # TODO: handle this!! This is NOT quite right....
        for struct in viable_structs:
            for remaining_player in group:
                if remaining_player == player:
                    continue
                print('p:', remaining_player)
                print(new_struct_to_ranking[struct])
                #new_struct_to_ranking[frozenset(new_group)][remaining_player] -= v(group)
        print('new_struct_to_ranking:', new_struct_to_ranking)
        best, worst = best_worst_player(player.name, viable_structs, new_struct_to_ranking) 
        print('>> best, worst LEAVE:', best, worst, 'for player:', player.name, '\n')
        best_worst[0, i] = best
        best_worst[1, i] = worst 
    return best_worst

In [56]:
group = frozenset({A, C})
assume_player_leave(group, struct_to_ranking)

viable structs: [(frozenset({B, C}), frozenset({A})), (frozenset({C}), frozenset({A}), frozenset({B}))]
p: A
OrderedDict([('A', 3), ('C', 6), ('B', 7)])
p: A
OrderedDict([('C', 1), ('B', 2), ('A', 3)])
new_struct_to_ranking: {(frozenset({C, B, A}),): OrderedDict([('C', 6), ('B', 7), ('A', 8)]), (frozenset({B, C}), frozenset({A})): OrderedDict([('A', 3), ('C', 6), ('B', 7)]), (frozenset({C, A}), frozenset({B})): OrderedDict([('B', 2), ('C', 6), ('A', 8)]), (frozenset({C}), frozenset({B, A})): OrderedDict([('C', 1), ('B', 7), ('A', 8)]), (frozenset({C}), frozenset({A}), frozenset({B})): OrderedDict([('C', 1), ('B', 2), ('A', 3)])}
ranking: OrderedDict([('A', 3), ('C', 6), ('B', 7)])
outcome: 1 for player: C with struct: (frozenset({B, C}), frozenset({A})) 

ranking: OrderedDict([('C', 1), ('B', 2), ('A', 3)])
outcome: 0 for player: C with struct: (frozenset({C}), frozenset({A}), frozenset({B})) 

>> best, worst LEAVE: 1 0 for player: C 

viable structs: [(frozenset({C}), frozenset({B, A}

array([[1., 2.],
       [0., 2.]])

Indeed, group $\{A, C\}$ is stable!

In [16]:
group = frozenset({A, C})
is_group_stable(group, struct_to_ranking)

viable staying: [(frozenset({A, C}), frozenset({B}))] 

ranking: OrderedDict([('B', 2), ('C', 6), ('A', 8)])
outcome: 2 for player: A with struct: (frozenset({A, C}), frozenset({B})) 

>> best, worst STAY: 2 2 for player: A 

ranking: OrderedDict([('B', 2), ('C', 6), ('A', 8)])
outcome: 1 for player: C with struct: (frozenset({A, C}), frozenset({B})) 

>> best, worst STAY: 1 1 for player: C 

ranking: OrderedDict([('C', 1), ('B', 7), ('A', 8)])
outcome: 2 for player: A with struct: (frozenset({C}), frozenset({A, B})) 

ranking: OrderedDict([('C', 1), ('B', 2), ('A', 3)])
outcome: 2 for player: A with struct: (frozenset({C}), frozenset({B}), frozenset({A})) 

>> best, worst LEAVE: 2 2 for player: A 

ranking: OrderedDict([('A', 3), ('C', 6), ('B', 7)])
outcome: 1 for player: C with struct: (frozenset({B, C}), frozenset({A})) 

ranking: OrderedDict([('C', 1), ('B', 2), ('A', 3)])
outcome: 0 for player: C with struct: (frozenset({C}), frozenset({B}), frozenset({A})) 

>> best, worst LEAVE

array(['N/A', 'N/A'], dtype='<U3')

Group $\{A, B\}$ is NOT stable!

In [17]:
group = frozenset({A, B})
is_group_stable(group, struct_to_ranking)

viable staying: [(frozenset({C}), frozenset({A, B}))] 

ranking: OrderedDict([('C', 1), ('B', 7), ('A', 8)])
outcome: 2 for player: A with struct: (frozenset({C}), frozenset({A, B})) 

>> best, worst STAY: 2 2 for player: A 

ranking: OrderedDict([('C', 1), ('B', 7), ('A', 8)])
outcome: 1 for player: B with struct: (frozenset({C}), frozenset({A, B})) 

>> best, worst STAY: 1 1 for player: B 

ranking: OrderedDict([('B', 2), ('C', 6), ('A', 8)])
outcome: 2 for player: A with struct: (frozenset({A, C}), frozenset({B})) 

ranking: OrderedDict([('C', 1), ('B', 2), ('A', 3)])
outcome: 2 for player: A with struct: (frozenset({C}), frozenset({B}), frozenset({A})) 

>> best, worst LEAVE: 2 2 for player: A 

ranking: OrderedDict([('A', 3), ('C', 6), ('B', 7)])
outcome: 2 for player: B with struct: (frozenset({B, C}), frozenset({A})) 

ranking: OrderedDict([('C', 1), ('B', 2), ('A', 3)])
outcome: 1 for player: B with struct: (frozenset({C}), frozenset({B}), frozenset({A})) 

>> best, worst LEAVE

array(['N/A', 'N/A'], dtype='<U3')

This indicates that it's better for player $B$ to actually switch! In the best case, it's better for $B$ to leave! 

## 4-player Game

In [18]:
A = Player('A', 3)
B = Player('B', 2)
C = Player('C', 1)
D = Player('D', 0)

players = [A, B, C, D]
all_partitions = get_partitions(players)
all_partitions

[(frozenset({D, C, B, A}),),
 (frozenset({D, C, B}), frozenset({A})),
 (frozenset({D, C, A}), frozenset({B})),
 (frozenset({D, C}), frozenset({B, A})),
 (frozenset({D, C}), frozenset({B}), frozenset({A})),
 (frozenset({D, B, A}), frozenset({C})),
 (frozenset({D, B}), frozenset({C, A})),
 (frozenset({D, B}), frozenset({C}), frozenset({A})),
 (frozenset({D, A}), frozenset({C, B})),
 (frozenset({D, A}), frozenset({C}), frozenset({B})),
 (frozenset({D}), frozenset({C, B, A})),
 (frozenset({D}), frozenset({C, B}), frozenset({A})),
 (frozenset({D}), frozenset({B, A}), frozenset({C})),
 (frozenset({D}), frozenset({B}), frozenset({C, A})),
 (frozenset({D}), frozenset({B}), frozenset({C}), frozenset({A}))]

In [19]:
struct_to_ranking = get_outcomes_of_each_structure(all_partitions)
struct_to_ranking

{(frozenset({D, C, B, A}),): OrderedDict([('D', 5),
              ('C', 6),
              ('B', 7),
              ('A', 8)]),
 (frozenset({D, C, B}),
  frozenset({A})): OrderedDict([('A', 3), ('D', 5), ('C', 6), ('B', 7)]),
 (frozenset({D, C, A}),
  frozenset({B})): OrderedDict([('B', 2), ('D', 5), ('C', 6), ('A', 8)]),
 (frozenset({D, C}),
  frozenset({B, A})): OrderedDict([('D', 5), ('C', 6), ('B', 7), ('A', 8)]),
 (frozenset({D, C}),
  frozenset({B}),
  frozenset({A})): OrderedDict([('B', 2), ('A', 3), ('D', 5), ('C', 6)]),
 (frozenset({D, B, A}),
  frozenset({C})): OrderedDict([('C', 1), ('D', 5), ('B', 7), ('A', 8)]),
 (frozenset({D, B}),
  frozenset({C, A})): OrderedDict([('D', 5), ('C', 6), ('B', 7), ('A', 8)]),
 (frozenset({D, B}),
  frozenset({C}),
  frozenset({A})): OrderedDict([('C', 1), ('A', 3), ('D', 5), ('B', 7)]),
 (frozenset({D, A}),
  frozenset({C, B})): OrderedDict([('D', 5), ('C', 6), ('B', 7), ('A', 8)]),
 (frozenset({D, A}),
  frozenset({C}),
  frozenset({B})): Or

In [20]:
group = frozenset({A, D})
is_group_stable(group, struct_to_ranking)

viable staying: [(frozenset({A, D}), frozenset({B, C})), (frozenset({A, D}), frozenset({C}), frozenset({B}))] 

ranking: OrderedDict([('D', 5), ('C', 6), ('B', 7), ('A', 8)])
outcome: 3 for player: A with struct: (frozenset({A, D}), frozenset({B, C})) 

ranking: OrderedDict([('C', 1), ('B', 2), ('D', 5), ('A', 8)])
outcome: 3 for player: A with struct: (frozenset({A, D}), frozenset({C}), frozenset({B})) 

>> best, worst STAY: 3 3 for player: A 

ranking: OrderedDict([('D', 5), ('C', 6), ('B', 7), ('A', 8)])
outcome: 0 for player: D with struct: (frozenset({A, D}), frozenset({B, C})) 

ranking: OrderedDict([('C', 1), ('B', 2), ('D', 5), ('A', 8)])
outcome: 2 for player: D with struct: (frozenset({A, D}), frozenset({C}), frozenset({B})) 

>> best, worst STAY: 2 0 for player: D 

ranking: OrderedDict([('D', 0), ('C', 6), ('B', 7), ('A', 8)])
outcome: 3 for player: A with struct: (frozenset({D}), frozenset({A, C, B})) 

ranking: OrderedDict([('D', 0), ('A', 3), ('C', 6), ('B', 7)])
outcome

array(['N/A', 'N/A'], dtype='<U3')

In [21]:
# 5 players
A = Player('A', 3)
B = Player('B', 2)
C = Player('C', 1)
D = Player('D', 0)
E = Player('E', -1)

players = [A, B, C, D, E]
all_partitions = get_partitions(players)
all_partitions

[(frozenset({E, D, C, B, A}),),
 (frozenset({E, D, C, B}), frozenset({A})),
 (frozenset({E, D, C, A}), frozenset({B})),
 (frozenset({E, D, C}), frozenset({B, A})),
 (frozenset({E, D, C}), frozenset({A}), frozenset({B})),
 (frozenset({E, D, B, A}), frozenset({C})),
 (frozenset({E, D, B}), frozenset({C, A})),
 (frozenset({E, D, B}), frozenset({A}), frozenset({C})),
 (frozenset({E, D, A}), frozenset({C, B})),
 (frozenset({E, D, A}), frozenset({C}), frozenset({B})),
 (frozenset({E, D}), frozenset({C, B, A})),
 (frozenset({E, D}), frozenset({C, A}), frozenset({B})),
 (frozenset({E, D}), frozenset({B, A}), frozenset({C})),
 (frozenset({E, D}), frozenset({A}), frozenset({C, B})),
 (frozenset({E, D}), frozenset({A}), frozenset({C}), frozenset({B})),
 (frozenset({E, C, B, A}), frozenset({D})),
 (frozenset({E, C, B}), frozenset({D, A})),
 (frozenset({E, C, B}), frozenset({A}), frozenset({D})),
 (frozenset({E, C, A}), frozenset({D, B})),
 (frozenset({E, C, A}), frozenset({D}), frozenset({B})),
 (

In [22]:
struct_to_ranking = get_outcomes_of_each_structure(all_partitions)
struct_to_ranking

{(frozenset({E, D, C, B, A}),): OrderedDict([('E', 4),
              ('D', 5),
              ('C', 6),
              ('B', 7),
              ('A', 8)]),
 (frozenset({E, D, C, B}),
  frozenset({A})): OrderedDict([('A', 3),
              ('E', 4),
              ('D', 5),
              ('C', 6),
              ('B', 7)]),
 (frozenset({E, D, C, A}),
  frozenset({B})): OrderedDict([('B', 2),
              ('E', 4),
              ('D', 5),
              ('C', 6),
              ('A', 8)]),
 (frozenset({E, D, C}),
  frozenset({B, A})): OrderedDict([('E', 4),
              ('D', 5),
              ('C', 6),
              ('B', 7),
              ('A', 8)]),
 (frozenset({E, D, C}),
  frozenset({A}),
  frozenset({B})): OrderedDict([('B', 2),
              ('A', 3),
              ('E', 4),
              ('D', 5),
              ('C', 6)]),
 (frozenset({E, D, B, A}),
  frozenset({C})): OrderedDict([('C', 1),
              ('E', 4),
              ('D', 5),
              ('B', 7),
              ('A', 8)]

In [23]:
group = frozenset({A, E})
is_group_stable(group, struct_to_ranking)

viable staying: [(frozenset({E, A}), frozenset({B, C, D})), (frozenset({E, A}), frozenset({C, D}), frozenset({B})), (frozenset({E, A}), frozenset({B, D}), frozenset({C})), (frozenset({E, A}), frozenset({D}), frozenset({B, C})), (frozenset({E, A}), frozenset({D}), frozenset({C}), frozenset({B}))] 

ranking: OrderedDict([('E', 4), ('D', 5), ('C', 6), ('B', 7), ('A', 8)])
outcome: 0 for player: E with struct: (frozenset({E, A}), frozenset({B, C, D})) 

ranking: OrderedDict([('B', 2), ('E', 4), ('D', 5), ('C', 6), ('A', 8)])
outcome: 1 for player: E with struct: (frozenset({E, A}), frozenset({C, D}), frozenset({B})) 

ranking: OrderedDict([('C', 1), ('E', 4), ('D', 5), ('B', 7), ('A', 8)])
outcome: 1 for player: E with struct: (frozenset({E, A}), frozenset({B, D}), frozenset({C})) 

ranking: OrderedDict([('D', 0), ('E', 4), ('C', 6), ('B', 7), ('A', 8)])
outcome: 1 for player: E with struct: (frozenset({E, A}), frozenset({D}), frozenset({B, C})) 

ranking: OrderedDict([('D', 0), ('C', 1), 

array(['N/A', 'N/A'], dtype='<U3')